# Bank Account

This section aims to show the real power of the Julia type system in combination with multiple dispatch. We will present it through an example where the goal is to create a structure that represents a bank account with the following properties:

- The structure has two fields: `owner` and `transaction`.
- It is possible to make transactions in different currencies.
- All transactions are stored in the currency in which they were made.

Before creating such a structure, we first define an abstract type `Currency` and its two concrete subtypes.

In [ ]:
abstract type Currency end

struct Euro <: Currency
    value::Float64
end

struct Dollar <: Currency
    value::Float64
end

Since `Euro` and `Dollar` are concrete types, we can create their instances and use `isa` to check that these instances are subtypes of `Currency`.

In [ ]:
Euro(1)
isa(Dollar(2), Currency) # equivalent to typeof(Dollar(2)) <: Currency

As `Currency` is an abstract type, we cannot create its instance. However, abstract types allow us to define generic functions that work for all their subtypes. We do so and define the `BankAccount` composite type.

In [ ]:
struct BankAccount{C<:Currency}
    owner::String
    transaction::Vector{Currency}

    function BankAccount(owner::String, C::Type{<:Currency})
        return new{C}(owner, Currency[C(0)])
    end
end

We will explain this type after creating its instance with the euro currency.

In [ ]:
b = BankAccount("Paul", Euro)

First, we observe that we use the `Euro` type (and not its instance) to instantiate the `BankAccount` type. The reason is the definition of the inner constructor for `BankAccount`, where the type annotation is `::Type{<:Currency}`. This is in contrast with `::Currency`. The former requires that the argument is a type, while the latter needs an instance.

Second, `BankAccount` is a parametric type, as can be seen from `BankAccount{Euro}`. In our example, this parameter plays the role of the primary account currency.

Third, due to the line `Currency[C(0)]` in the inner constructor, transactions are stored in a vector of type `Vector{Currency}`. The expression `C(0)` creates an instance of the currency `C` with zero value. The `Currency` type combined with the square brackets creates a vector that may contain instances of any subtypes of `Currency`. It is, therefore, possible to push a new transaction in a different currency to the `transaction` field.

In [ ]:
push!(b.transaction, Dollar(2))
b

It is crucial to use `Currency` in `Currency[C(0)]`. Without it, we would create an array of type `C` only. We would not be able to add transactions in different currencies to this array as Julia could not convert the different currencies to `C`.

In [ ]:
w = [Euro(0)]
push!(w, Dollar(2))

We used only the abstract type `Currency` to define the `BankAccount` type. This allows us to write generic code that is not constrained to one concrete type. We created an instance of `BankAccount` and added a new transaction. However, we cannot calculate an account balance (the sum of all transactions), and we cannot convert money from one currency to another. In the rest of the lecture, we will fix this, and we will also define basic arithmetic operations such as `+` or `-`.

> **Note:** It is generally not good to use [containers with abstract element type](https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-abstract-container) as we did for storing transactions. We used it in the example above because we do not want to convert all transactions to a common currency. When we create an array from different types, the promotion system converts these types to their smallest supertype for efficient memory storage.

```julia
[Int32(123), 1, 1.5, 1.234f0]
```

The smallest supertype is `Float64`, and the result is `Array{Float64, 1}`. When we do not want to convert the variables, we must manually specify the resulting array supertype.

```julia
Real[Int32(123), 1, 1.5, 1.234f0]
```

In this case, the types of all elements are preserved.

## Custom Print

Each currency has its symbol, such as € for the euro. We will redefine the `show` function to print the currency in a prettier way. First, we define a new function `symbol` that returns the used currency symbol.

In [ ]:
symbol(T::Type{<:Currency}) = string(nameof(T))
symbol(::Type{Euro}) = "€"

We defined one method for all subtypes of `Currency` and one method for the `Euro` type. With the `symbol` function, we can define nicer printing by adding a new method to the `show` function from `Base`. It is possible to define a custom show function for different output formats. For example, it is possible to define different formatting for HTML output. The example below shows only basic usage; for more information, see the [official documentation](https://docs.julialang.org/en/v1/manual/types/#man-custom-pretty-printing).

```julia
Base.show(io::IO, c::C) where {C <: Currency} = print(io, c.value, " ", symbol(C))
```

In [ ]:
Base.show(io::IO, c::C) where {C <: Currency} = print(io, c.value, " ", symbol(C))

Euro(1)
Euro(1.5)

### Exercise:

Define a new method for the `symbol` function for `Dollar`.

**Hint:** The dollar symbol `$` has a special meaning in Julia. To include it in a string, you can escape it with a backslash `\` or use a raw string.

<details>
<summary><strong>Solution:</strong></summary>

When adding a new method to the `symbol` function, we have to remember that we used the currency type for dispatch, i.e., we have to use `::Type{Dollar}` instead of `::Dollar` in the type annotation.

```julia
symbol(::Type{Dollar}) = "\$"
```

Now we can check that everything works well.

```julia
Dollar(1)
Dollar(1.5)
```

</details>

In [ ]:
symbol(::Type{Dollar}) = "\$"

Dollar(1)
Dollar(1.5)

## Conversion

In the previous section, we have defined two currencies. A natural question is how to convert one currency to the other. In the real world, the exchange operation between currencies is not transitive. However, we assume that the **exchange rate is transitive** and there are no exchange costs.

The simplest way to define conversions between the currencies is to define the conversion function for each pair of currencies. This can be done efficiently only for two currencies.

In [ ]:
dollar2euro(c::Dollar) = Euro(0.83 * c.value)
euro2dollar(c::Euro) = Dollar(c.value / 0.83)

We can check that the result is correct.

In [ ]:
eur = dollar2euro(Dollar(1.3))
euro2dollar(eur)

Even though this is a way to write code, there is a more general way. We start with a conversion rate between two types.

In [ ]:
rate(::Type{Euro}, ::Type{Dollar}) = 0.83

Transitivity implies that if one exchange rate is $r_{1 \rightarrow 2}$, the opposite exchange rate equals $r_{2 \rightarrow 1} = r_{1 \rightarrow 2}^{-1}$. We create a generic function to define the exchange rate in the opposite direction.

In [ ]:
rate(T::Type{<:Currency}, ::Type{Euro}) = 1 / rate(Euro, T)

If we use only the two methods above, it computes the exchange rate between `Dollar` and `Euro`.

In [ ]:
rate(Euro, Dollar)
rate(Dollar, Euro)

However, the definition is not complete because the `rate` function does not work if we use the same currencies.

In [ ]:
rate(Euro, Euro)
rate(Dollar, Dollar)

To solve this issue, we have to add two new methods. The first one defines that the exchange rate between the same currency is `1`.

In [ ]:
rate(::Type{T}, ::Type{T}) where {T<:Currency} = 1

This method solves the issue for the `Dollar` to `Dollar` conversion.

In [ ]:
rate(Dollar, Dollar)

However, it does not solve the problem with `Euro` to `Euro` conversion.

In [ ]:
rate(Euro, Euro)

To fix this, we add a specific method for `Euro` to `Euro`.

In [ ]:
rate(::Type{Euro}, ::Type{Euro}) = 1

This method solves the issue, as can be seen in the example below.

In [ ]:
rate(Euro, Euro)

The transitivity also implies that instead of converting the `C1` currency directly to the `C2` currency, we can convert it to some `C` and then convert `C` to `C2`. In our case, we use the `Euro` as the intermediate currency. When adding a new currency, it suffices to specify its exchange rate only to the euro.

In [ ]:
rate(T::Type{<:Currency}, C::Type{<:Currency}) = rate(Euro, C) * rate(T, Euro)

To test the `rate` function, we add a new currency.

In [ ]:
struct Pound <: Currency
    value::Float64
end

symbol(::Type{Pound}) = "£"
rate(::Type{Euro}, ::Type{Pound}) = 1.13

We can quickly test that the `rate` function works in all possible cases correctly in the following way.

In [ ]:
rate(Pound, Pound) # 1
rate(Euro, Pound) # 1.13
rate(Pound, Euro) # 1/1.13
rate(Dollar, Pound) # 1.13 * 1/0.83
rate(Pound, Dollar) # 0.83 * 1/1.13

We have defined the `rate` function with all necessary methods. To convert currency types, we need to extend the `convert` function from `Base` by the following two methods:

In [ ]:
Base.convert(::Type{T}, c::T) where {T<:Currency} = c
Base.convert(::Type{T}, c::C) where {T<:Currency, C<:Currency} = T(c.value * rate(T, C))

Finally, we test that the `convert` function indeed converts its input to a different type.

In [ ]:
eur = convert(Euro, Dollar(1.3))
pnd = convert(Pound, eur)
dlr = convert(Dollar, pnd)

### Exercise:

The printing style is not ideal because we are usually not interested in more than the first two digits after the decimal point. Redefine the method in the `show` function to print currencies so that the result is rounded to 2 digits after the decimal point.

<details>
<summary><strong>Solution:</strong></summary>

Any real number can be rounded to 2 digits after the decimal point by the `round` function with the keyword argument `digits = 2`. Then we can use an almost identical definition of the method as before.

```julia
function Base.show(io::IO, c::T) where {T <: Currency}
    val = round(c.value; digits = 2)
    return print(io, val, " ", symbol(T))
end
```

The same code as before this example gives the following results.

```julia
eur = convert(Euro, Dollar(1.3))
pnd = convert(Pound, eur)
dlr = convert(Dollar, pnd)
```

We realize that the rounding is done only for printing, while the original value remains unchanged.

</details>

In [ ]:
function Base.show(io::IO, c::T) where {T <: Currency}
    val = round(c.value; digits = 2)
    return print(io, val, " ", symbol(T))
end

eur = convert(Euro, Dollar(1.3))
pnd = convert(Pound, eur)
dlr = convert(Dollar, pnd)

## Promotion

Before defining basic arithmetic operations for currencies, we have to decide how to work with money in different currencies. Imagine that we want to add `1€` and `1$`. Should the result be euro or dollar? For such a situation, Julia provides a promotion system that allows defining simple rules for promoting custom types. The promotion system can be modified by defining custom methods for the `promote_rule` function. For example, the following definition means that the euro has precedence against all other currencies.

In [ ]:
Base.promote_rule(::Type{Euro}, ::Type{<:Currency}) = Euro

Since we have three different currencies, we also define the promotion type for the pair `Dollar` and `Pound`.

In [ ]:
Base.promote_rule(::Type{Dollar}, ::Type{Pound}) = Dollar

The `promote_rule` function is used as a building block for the `promote_type` function, which returns the promoted type of inputs.

In [ ]:
promote_type(Euro, Dollar)
promote_type(Pound, Dollar)
promote_type(Pound, Dollar, Euro)

When we have instances instead of types, we can use the `promote` function to convert them to their representation in the promoted type.

In [ ]:
promote(Euro(2), Dollar(2.4))
promote(Pound(1.3), Euro(2))
promote(Pound(1.3), Dollar(2.4), Euro(2))

### Exercise:

Define a new currency `CzechCrown` representing Czech crowns. The exchange rate to euro is `0.038`, and all other currencies should take precedence over the Czech crown.

<details>
<summary><strong>Solution:</strong></summary>

We define first the new type `CzechCrown`.

```julia
struct CzechCrown <: Currency
    value::Float64
end
```

We must add new methods for the `symbol` and `rate` functions.

```julia
symbol(::Type{CzechCrown}) = "Kč"
rate(::Type{Euro}, ::Type{CzechCrown}) = 0.038
```

We also must add promotion rules for the dollar and pound.

```julia
Base.promote_rule(::Type{CzechCrown}, ::Type{Dollar}) = Dollar
Base.promote_rule(::Type{CzechCrown}, ::Type{Pound}) = Pound
```

Finally, we can test the functionality.

```julia
CzechCrown(2.8)
dl = convert(Dollar, CzechCrown(64))
convert(CzechCrown, dl)
promote(Pound(1.3), Dollar(2.4), Euro(2), CzechCrown(2.8))
```

</details>

In [ ]:
struct CzechCrown <: Currency
    value::Float64
end

symbol(::Type{CzechCrown}) = "Kč"
rate(::Type{Euro}, ::Type{CzechCrown}) = 0.038

Base.promote_rule(::Type{CzechCrown}, ::Type{Dollar}) = Dollar
Base.promote_rule(::Type{CzechCrown}, ::Type{Pound}) = Pound

CzechCrown(2.8)
dl = convert(Dollar, CzechCrown(64))
convert(CzechCrown, dl)
promote(Pound(1.3), Dollar(2.4), Euro(2), CzechCrown(2.8))

## Basic Arithmetic Operations

Now we are ready to define basic arithmetic operations. As usual, we can do this by adding a new method to standard functions. We start with the addition, where there are two cases to consider. The first one is the summation of two different currencies. In this case, we use the `promote` function to convert these two currencies to their promoted type.

In [ ]:
Base.:+(x::Currency, y::Currency) = +(promote(x, y)...)

The second one is the summation of the same currency. In this case, we know the resulting currency, and we can sum the `value` fields.

In [ ]:
Base.:+(x::T, y::T) where {T <: Currency} = T(x.value + y.value)

Now we can sum money in different currencies.

In [ ]:
Dollar(1.3) + CzechCrown(4.5)
CzechCrown(4.5) + Euro(3.2) + Pound(3.6) + Dollar(12)

Moreover, we can use, for example, the `sum` function without any additional changes.

In [ ]:
sum([CzechCrown(4.5), Euro(3.2), Pound(3.6), Dollar(12)])

Also, the broadcasting works natively for arrays of currencies.

In [ ]:
CzechCrown.([4.5, 2.4, 16.7, 18.3]) .+ Pound.([1.2, 2.6, 0.6, 1.8])

However, there is a problem if we want to sum a vector of currencies with one currency. In such a case, an error will occur.

In [ ]:
CzechCrown.([4.5, 2.4, 16.7, 18.3]) .+ Dollar(12)

The reason is that Julia assumes that custom structures are iterable. But in our case, all subtypes of the `Currency` type represent scalar values. This situation can be easily fixed by defining a new method to the `broadcastable` function from `Base`.

In [ ]:
Base.broadcastable(c::Currency) = Ref(c)

Now we can test if the broadcasting works as expected.

In [ ]:
CzechCrown.([4.5, 2.4, 16.7, 18.3]) .+ Dollar(12)

### Exercise:

In the section above, we defined the addition for all subtypes of `Currency`. We also told the broadcasting system in Julia to treat all subtypes of the `Currency` as scalars. Follow the same pattern and define the following operations: `-`, `*`, `/`.

**Hint:** Define only operations that make sense. For example, it makes sense to multiply `1 €` by 2 to get `2 €`. But it does not make sense to multiply `1 €` by `2 €`.

<details>
<summary><strong>Solution:</strong></summary>

The `-` operation can be defined exactly as the addition.

```julia
Base.:-(x::Currency, y::Currency) = -(promote(x, y)...) 
Base.:-(x::T, y::T) where {T <: Currency} = T(x.value - y.value)
```

The multiplication makes sense when multiplying a currency by a real number.

```julia
Base.:*(a::Real, x::T) where {T <: Currency} = T(a * x.value)
Base.:*(x::T, a::Real) where {T <: Currency} = T(a * x.value)
```

Division can be defined similarly.

```julia
Base.:/(x::T, a::Real) where {T <: Currency} = T(x.value / a)
```

But it also makes sense to define the division of one amount of money by another amount of money in different currencies. In this case, the result is a real number representing their ratio.

```julia
Base.:/(x::Currency, y::Currency) = /(promote(x, y)...) 
Base.:/(x::T, y::T) where {T <: Currency} = x.value / y.value
```

</details>

In [ ]:
Base.:-(x::Currency, y::Currency) = -(promote(x, y)...) 
Base.:-(x::T, y::T) where {T <: Currency} = T(x.value - y.value)

In [ ]:
Base.:*(a::Real, x::T) where {T <: Currency} = T(a * x.value)
Base.:*(x::T, a::Real) where {T <: Currency} = T(a * x.value)

In [ ]:
Base.:/(x::T, a::Real) where {T <: Currency} = T(x.value / a)

Base.:/(x::Currency, y::Currency) = /(promote(x, y)...) 
Base.:/(x::T, y::T) where {T <: Currency} = x.value / y.value

## Currency Comparison

The last thing we should define is comparison operators. To provide full functionality, we have to add new methods to two functions. The first one is the value equality operator `==`. By default, it uses the following definition `==(x, y) = x === y`. The `===` operator determines whether `x` and `y` are identical, in the sense that no program could distinguish them.

In [ ]:
Dollar(1) == Euro(0.83)
Dollar(1) != Euro(0.83)

To allow this kind of comparison, we can define new methods to the `==` function as follows:

In [ ]:
Base.:(==)(x::Currency, y::Currency) = ==(promote(x, y)...) 
Base.:(==)(x::T, y::T) where {T <: Currency} = ==(x.value, y.value)

With these two methods defined, the comparison works as expected.

In [ ]:
Dollar(1) == Euro(0.83)
Dollar(1) != Euro(0.83)

The second function to extend is the `isless` function. In this case, the logic is the same as before: We want to compare values stored in the structure.

In [ ]:
Base.isless(x::Currency, y::Currency) = isless(promote(x, y)...) 
Base.isless(x::T, y::T) where {T <: Currency} = isless(x.value, y.value)

As can be seen below, all operations work as intended.

In [ ]:
Dollar(1) < Euro(0.83)
Dollar(1) > Euro(0.83)
Dollar(1) <= Euro(0.83)
Dollar(1) >= Euro(0.83)

Other functions based only on comparison will work for all subtypes of `Currency` without any additional changes. Examples include `extrema`, `argmin`, or `sort` functions.

In [ ]:
vals = Currency[CzechCrown(100), Euro(0.83), Pound(3.6), Dollar(1.2)]
extrema(vals)
argmin(vals)
sort(vals)

## Back to the Bank Account

In the previous sections, we defined all the functions and types needed for the `BankAccount` type and performed basic arithmetic and other operations on currencies.  For a bank account, we are primarily interested in its balance. Since we store all transactions in a vector, the account balance can be computed as a sum of the `transaction` field.

In [ ]:
balance(b::BankAccount{C}) where {C} = convert(C, sum(b.transaction))

We convert the balance to the primary currency of the account.

In [ ]:
b = BankAccount("Paul", CzechCrown)
balance(b)

Another thing that we can define is custom pretty-printing.

In [ ]:
function Base.show(io::IO, b::BankAccount{C}) where {C<:Currency}
    println(io, "Bank Account:")
    println(io, "  - Owner: ", b.owner)
    println(io, "  - Primary currency: ", nameof(C))
    println(io, "  - Balance: ", balance(b))
    print(io,   "  - Number of transactions: ", length(b.transaction))
end

The previous method definition results in the following output.

In [ ]:
b

The last function that we define is the function that adds a new transaction into the given bank account. Even though it can be defined like any other function, we decided to use a special syntax. Since methods are associated with types, making any arbitrary Julia object "callable" is possible by adding methods to its type. Such "callable" objects are sometimes called "functors".

In [ ]:
function (b::BankAccount{T})(c::Currency) where {T}
    balance(b) + c >= T(0) || throw(ArgumentError("Insufficient bank account balance."))
    push!(b.transaction, c)
    return
end

The first thing in the function above is the check whether there is a sufficient account balance. If not, the function will throw an error. Otherwise, the function will push a new element to the `transaction` field.

In [ ]:
b(Dollar(10))
b(-2 * balance(b))
b(Pound(10))
b(Euro(23.6))
b(CzechCrown(152))
b

Note that all transactions are stored in their original currency, as can be seen if we print the `transaction` field.

In [ ]:
b.transaction